In [1]:
import numpy as np
import pandas as pd
import copy

from hongxiongmao import utilities
from hongxiongmao import download
from hongxiongmao import overplot
dl = download.quandl_hxm()

import plotly.offline as py
import plotly_express as pyex
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

In [2]:
cli = dl.from_tickerdict('oecd_cli', start_date='-20y') - 100
gdp = dl.from_tickerdict('oecd_normalised_gdp', start_date='-20y') - 100

In [3]:
df = pd.concat([cli.loc[:,'US'], gdp.loc[:,'US']], axis=1, join='inner')
df.columns = ['cli', 'ref']
df.tail()

,cli,ref
Date,,
2018-12-31,-0.48648,0.2844
2019-01-31,-0.67959,0.2990
2019-02-28,-0.82941,0.3137
2019-03-31,-0.93383,0.3271
2019-04-30,-1.00853,NaN


def swirlygram_animated(df, n=3, lead=1, trail=12, animation=True):
    """
    Animated Swirlygram Function
    
    Swirlygrams are designed to show the amplitude (above or below zero) vs. the periodic change. The swirl coming from a
    timeseries tail. For the animated version we show the swirlygram through time.

    This function looks at a specific case - originally to show the OECD CLI (lagged) vs. OECD Normalised GDP. 
    Function takes a timeseries dataframe with col 0 as a leading indicator & col 1 as the reference variable.
    We then create two reference dataframes which are timeseries of x & y for the CLI and the Ref series, incorporating 
    the lag variable.
    
    NB/ This is a stepping stone to create the more general case... ideally with buttons for multiple plots at the same time
    
    """
    
    ### Default Setup
    cmap=overplot.COLOURMAP.copy()
    layout=copy.deepcopy(overplot.DEFAULT_LAYOUT)
    layout=overplot._update_layout(layout, xaxis={'autorange':False}, yaxis={'autorange':False})
    data=[]
    frames=[]
    
        
    ### Data Manipulation
    
    # Subset Leading Indicator & Reference Index
    # Each has columns x (change) & y (absolute)
    cli = pd.DataFrame(data={'x':df.iloc[:,0]-df.iloc[:,0].shift(n),'y':df.iloc[:,0]}).shift(lead)
    ref = pd.DataFrame(data={'x':df.iloc[:,1]-df.iloc[:,1].shift(n),'y':df.iloc[:,1]})
    
    # Remove stuff
    
    ### Basic Traces
                
    # MARKER for CLI
    data.append(dict(type='scatter', name='lead', mode='markers', showlegend=False, 
                     x=[cli['x'].iloc[-1]],
                     y=[cli['y'].iloc[-1]],
                     marker=dict(symbol='diamond', color=cmap[0], size=10, line={'color':'black', 'width':1}),
                     ))
    
    # MARKER for Ref
    data.append(dict(type='scatter', name='lead', mode='markers', showlegend=False, 
                     x=[ref['x'].iloc[-1]],
                     y=[ref['y'].iloc[-1]],
                     marker=dict(symbol='diamond', color=cmap[1], size=10, line={'color':'black', 'width':1}),
                     ))
    
    # LINE CLI
    data.append(dict(type='scatter', name='lead_line', mode='lines+markers', showlegend=True,
                     x=cli['x'].iloc[-trail:], y=cli['y'].iloc[-trail:],
                     line=dict(color=cmap[0], width=1),
                     ))
    
    # LINE REF
    data.append(dict(type='scatter', name='ref_line', mode='lines+markers', showlegend=True,
                     x=ref['x'].iloc[-trail:], y=ref['y'].iloc[-trail:],
                     line=dict(color=cmap[1], width=1),
                     ))
    
    sliders = {'yanchor': 'top', 'xanchor': 'left', 
                           'currentvalue': {'font':{'size': 12}, 'prefix':'Date : ', 'xanchor': 'left'},
                           'transition': {'duration': 500, 'easing': 'linear'},
                           'pad': {'b': 0, 't': 25}, 'len': 1, 'x': 0, 'y': 0,
                           'steps':[]}
    
    
    ### Additional Layout Changes
    
    # Quadrants
    layout = overplot._quadrants(layout)
    
    # Symmetrical Plot around zero
    absmax = lambda c, x=cli, y=ref: pd.concat([x.iloc[:,c], y.iloc[:,c]]).abs().max()*1.05 
    layout['xaxis']['range'] = [-absmax(0), absmax(0)]
    layout['yaxis']['range'] = [-absmax(1), absmax(1)]
    
    ### Build Figure
    fig = dict(data=data, layout=layout)
    
    
    ### Animations
    if animation:
        
        # Play/Pause Buttons
        buttons = [{'label': 'Play', 'method': 'animate',
                    'args':[None, {'frame': {'duration': 10, 'redraw': False},
                                   'fromcurrent': True,
                                   'transition': {'duration': 10, 'easing': 'quadratic-in-out'}}],},
                   {'label': 'Pause', 'method': 'animate',
                    'args': [[None], {'frame': {'duration': 10, 'redraw': False},
                                      'mode': 'immediate',
                                      'transition': {'duration': 10}}],}]
        
        fig['layout']['updatemenus'] = [{'buttons': buttons,
                                         'direction': 'left',
                                         'pad': {'r': 0, 't': 0},
                                         'showactive': False,
                                         'type': 'buttons',
                                         'x': 0, 'xanchor': 'left',
                                         'y': 0, 'yanchor': 'bottom'
                                     }]
        
        # Iterate through cli adding data sets to frames for each step in animation
        for i, v in enumerate(cli.index.values[(trail+lead+n):-1]):
            
            label = pd.to_datetime(str(v)).strftime('%m/%y')
            #label = '{%m/%y}'.format()    # string label - used to link slider and frame (data things)

            # Append "frame" dictionary to frames list
            frames.append({'name':i, 'layout':{},
                           'data':[dict(type='scatter', x=[cli['x'].iloc[i]], y=[cli['y'].iloc[i]]), # Marker CLI
                                   dict(type='scatter', x=[ref['x'].iloc[i]], y=[ref['y'].iloc[i]]), # Marker Ref
                                   dict(type='scatter', x=cli['x'].iloc[i-trail+1:i],
                                                        y=cli['y'].iloc[i-trail+1:i]), # Line CLI
                                   dict(type='scatter', x=ref['x'].iloc[i-trail+1:i],
                                                        y=ref['y'].iloc[i-trail+1:i]),
                                  ]})

            # Append a "step" dictionary to steps list in sliders dict
            sliders['steps'].append({'label':label, 'method': 'animate', 
                                     'args':[[i], {'frame': {'duration': 250, 'easing':'linear', 'redraw':False},
                                                   'transition':{'duration': 100, 'easing': 'linear'}}],
                                    })

        fig['frames'] = frames
        fig['layout']['sliders'] = [sliders]        # Append completed sliders dictionary to layout
                
    return fig

fig = swirlygram(df, title='OECD Normalised CLI vs. GDP through time', animation=True)
py.iplot(fig, auto_play=False)